In [1]:
#!pip install pytesseract

/bin/sh: /home/kbeelen/.local/bin/pip: /data/anaconda/envs/py35/bin/python: bad interpreter: No such file or directory


In [1]:
from pathlib import Path
from PIL import Image
import pytesseract
import numpy as np
import pandas as pd
import nltk
from collections import defaultdict
from nltk.translate.bleu_score import sentence_bleu
from difflib import SequenceMatcher as SQ

In [3]:
import PIL
PIL.__version__

'7.1.2'

# Batch Evaluation

In [6]:
eval_folder = "/deezy_datadrive/kaspar-playground/tesseract/tesstrain/validation_data/validation"
path = Path(eval_folder)
files_root = set([p.stem.split('.')[0] for p in path.glob("**/*")])
print(f" There are {len(files_root)} items for evaluation.")

 There are 67 items for evaluation.


## Compare ABBYY to Fine-tuned tesseract

In [17]:
def evaluate_model(model_name, verbose = True):


    if model_name == 'eng':
        tessdata_path = "/deezy_datadrive/kaspar-playground/tesseract/tessdata"
    else:
        tessdata_path = "/deezy_datadrive/kaspar-playground/tesseract/tesstrain/data"

    PATH_TESSERACT,LANG = (tessdata_path, model_name)
    tessdata_dir_config = f'--tessdata-dir "{PATH_TESSERACT}" --psm 3'

    eval_folder = "/deezy_datadrive/kaspar-playground/tesseract/tesstrain/validation_data/validation"
    path = Path(eval_folder)
    files_root = set([p.stem.split('.')[0] for p in path.glob("**/*")])
    
    if verbose:
        print(f" There are {len(files_root)} items for evaluation.")

    tess_scores = []

    for f in files_root:
    
        #if not f:
        #    continue
    
        # load ground truth
        gt = (path / (f+".gt.txt")).read_text()
        # we'll ignore line splits kinda
        gt = ' '.join(gt.split('\n'))
    
        # now load the image and OCR it with fine-tuned tesseract
        # load image
        image = Image.open(path / (f+".png"))
        
        # apply the fine-tuned model
        tess_ft = pytesseract.image_to_string(image, lang=LANG, config=tessdata_dir_config)
        tess_ft = ' '.join(tess_ft.split('\n'))
    
        # compute accuracy of fine-tuned model
        tess_acc = round(SQ(None, tess_ft, gt).ratio()*100,2)
        tess_scores.append(tess_acc) # append scores
        #blue_scores.append([sentence_bleu([[ch for ch in gt]],[ch for ch in tess_ft]))

        if verbose:
            print('\n')
            print(f)
            print(f"SCORE={tess_acc}%")
            print(f'GROUND TRUTH:\n{gt}\n')
            print(f'TESSERACT FINE-TUNED:\n{tess_ft}\n')

    return np.array(tess_scores)



In [18]:
eng = evaluate_model('eng')
#npd_2 = evaluate_model('npd_2',verbose=False)
#npd_1 = evaluate_model('npd_1',verbose=False)
#npd = evaluate_model('npd',verbose=False)

#npd_1[npd_1 > 0].mean()


 There are 67 items for evaluation.


val_2
SCORE=96.3%
GROUND TRUTH:
BELFAST EVENING TELEGRAPH.

TESSERACT FINE-TUNED:
BELFAST EVENING TELEGRAPH. 



val_57
SCORE=81.82%
GROUND TRUTH:
Principles: Liberal. It is, however, chiefly confirmed

TESSERACT FINE-TUNED:
PrINcIPLES : Liberal. It is, however, chiefly confined 



val_15
SCORE=92.98%
GROUND TRUTH:
Its political articles are written with vigour, and a

TESSERACT FINE-TUNED:
Its political articles are written with oreat vigour, and a 



val_20
SCORE=97.73%
GROUND TRUTH:
MAIDSTONE AND KENTISH JOURNAL. Thursday and

TESSERACT FINE-TUNED:
MAIDSTONE AND KENTISH JOURNAL. Thursday and 



val_16
SCORE=95.0%
GROUND TRUTH:
NEWCASTLE EVENING CHRONICLE. Daily, ½d.

TESSERACT FINE-TUNED:
NEWCASTLE EVENING CHRONICLE. Daily, id. 



val_10
SCORE=83.61%
GROUND TRUTH:
Pinner, Northwood, Ruislip, Harrow Weald, Northolt,

TESSERACT FINE-TUNED:
eT a mere Tea wy  Pinner, Northwood, Ruislip, Harrow Weald, Northolt, 



val_40
SCORE=86.79%
GROUN

In [21]:
print(npd_2.mean())
print(npd_1.mean())
print(npd.mean())
print(eng.mean())

91.17134328358209
90.8544776119403
90.66149253731344
86.17014925373135
